In [1]:
# set theme
!jt -t oceans16

In [2]:
# this is to make the Jupyter notebook as wide as the screen (on the Mac at least)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%config InlineBackend.figure_format ='retina'

### Import needed modules

In [3]:
# import BBP_RTQC
import BBP_RTQC
from BBP_RTQC_paths import *
import subprocess
import numpy as np
import glob

In [11]:
# these are the tests and their codes
tests = {"A": "Global Range",
         "B": "Noisy Profile",
         "C": "High-Deep Value",
         "D": "Surface Hook", 
         "E": "Missing Data",
         "F": "Negative non-surface",
         "G": "Parking Hook"      
        }
egs = {  "A": "coriolis_BR7900591_240.nc",
         "A2": "coriolis_BR6903093_044.nc",
         "B": "",
         "C": "",
         "D": "coriolis_BD6901482_097.nc", 
         "E": "",
         "F": "",
         "G": "coriolis_BD6901580_107.nc"      
        }

In [4]:
# create a list (wmo_dac) and a numpy array (WMO_dac) of dac/wmo to process
# e.g., WMO_dac[0]='aoml/1902303'
cmdo2 = "grep BBP700 " + MAIN_DIR + "dac/argo_bio-profile_index.txt |   cut -d '/' -f1,2  | uniq "  # all data
wmo_dac = subprocess.run(cmdo2, shell=True, stdout=subprocess.PIPE).stdout.decode('utf-8').split('\n')
WMO_dac = np.asarray(wmo_dac[::])
print(len(WMO_dac))
istart = 0

614


In [5]:
# QC a single float
# BBP_RTQC.QC_wmo('aoml/5905105')

In [6]:
# # remove old pkl + png files (may not be necessary, but just to be sure)
# oldpkl = glob.glob(PKL_DIR + "/*.p*")
# [os.remove(i) for i in oldpkl] ;

In [7]:
# # this is to run the QC in parallel on multiple floats
# from joblib import delayed, Parallel
# r = Parallel(n_jobs=7, verbose=40)(delayed(BBP_RTQC.QC_wmo)(iwmo) for iwmo in WMO_dac[20::])


In [8]:
iwmo = 'aoml/5905372'
prof = '080'
QCstatus = 'BD'
dac = iwmo.split('/')[0]
wmo = iwmo.split('/')[-1]

# read meta file
[ds_config, SENSOR_MODEL, SENSOR_MAKER, SENSOR_SERIAL_NO, PLATFORM_TYPE, miss_no_float] = BBP_RTQC.rd_WMOmeta(iwmo, VERBOSE=False)

# list single profiles
fn2glob = MAIN_DIR + "dac/" + iwmo + "/profiles/" + "B*" + iwmo.split("/")[-1] + "*_[0-9][0-9][0-9].nc"
fn_single_profiles = np.sort(glob.glob(fn2glob))

# select specific profile to QC
fn_p = fn_single_profiles[0]
fn_p = MAIN_DIR + "dac/" + iwmo + "/profiles/" + QCstatus+wmo+'_'+prof+'.nc'
print(fn_p)
# read BBP and PRES + other data from profile file
[ PRES, BBP700, JULD, LAT, LON, BBP700mf1, miss_no_prof, PARK_PRES, maxPRES, innan] = BBP_RTQC.rd_BBP(fn_p, miss_no_float, ds_config)

/data/datasets/Argo/Original/dac/aoml/5905372/profiles/BD5905372_080.nc


In [9]:
# create BBP700_QC_flags, BBP700_QC_1st_failed_test
BBP700_QC_flags = np.zeros(BBP700.shape)+1
BBP700_QC_1st_failed_test = np.full(shape=BBP700.shape, fill_value='0')


In [10]:
# NOISY-PROFILE TEST for BBP700
BBP700_QC_flag, BBP700_QC_1st_failed_test, res = BBP_RTQC.BBP_Noisy_profile_test(BBP700, BBP700mf1, PRES,
                                                                  BBP700_QC_flags, BBP700_QC_1st_failed_test, 
                                                                  fn_p, VERBOSE=True, PLOT=True, SAVEPLOT=False)




In [11]:
# GLOBAL-RANGE TEST for BBP700
BBP700_QC_flag, BBP700_QC_1st_failed_test = BBP_RTQC.BBP_Global_range_test(BBP700, BBP700mf1, PRES, 
                                                                  BBP700_QC_flags, BBP700_QC_1st_failed_test, 
                                                                  fn_p, VERBOSE=True, PLOT=True, SAVEPLOT=False)


Failed Global_Range_test
applying QC=3...


In [ ]:
# SURFACE_HOOK TEST for BBP700
BBP700_QC_flag, BBP700_QC_1st_failed_test = BBP_RTQC.BBP_Surface_hook_test(BBP700, BBP700mf1, PRES, 
                                                                  BBP700_QC_flags, BBP700_QC_1st_failed_test, 
                                                                  fn_p, VERBOSE=True, PLOT=True, SAVEPLOT=False)




In [ ]:
# PARKING-HOOK TEST for BBP700
BBP700_QC_flag, BBP700_QC_1st_failed_test = BBP_RTQC.BBP_Parking_hook_test(BBP700, BBP700mf1, PRES,
                                                                  maxPRES, PARK_PRES,
                                                                  BBP700_QC_flags, BBP700_QC_1st_failed_test, 
                                                                  fn_p, VERBOSE=True, PLOT=True, SAVEPLOT=False)




In [ ]:
# NEGATIVE-NON-SURFACE TEST for BBP700
BBP700_QC_flag, BBP700_QC_1st_failed_test = BBP_RTQC.BBP_Negative_nonsurface_test(BBP700, PRES,
                                                                  BBP700_QC_flags, BBP700_QC_1st_failed_test, 
                                                                  fn_p, VERBOSE=True, PLOT=True, SAVEPLOT=False)


